In [9]:
from sklearn.base import BaseEstimator
import numpy as np

# BaseEstimator를 상속받아 Customized 형태의 Classifier 클래스 제작
class MyDummyClassifier(BaseEstimator):
    # fit() 메소드는 아무것도 하지 않음
    def fit(self, X, y=None):
        pass
    # predict() : Sex feature가 1이면 0, 그렇지 않으면 1로 예측하는 함수
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        
        return pred

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [11]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거 함수
def drop_unnecessary_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    
    return df

# 레이블 인코딩 수행 함수
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        encoder = LabelEncoder()
        encoder = encoder.fit(df[feature])
        df[feature] = encoder.transform(df[feature])
    
    return df

# 위 3개의 함수 호출하는 데이터 전처리 함수
def transform_features(df):
    df = fillna(df)
    df = drop_unnecessary_features(df)
    df = format_features(df)
    
    return df

In [12]:
# 데이터 로딩
titanic_df = pd.read_csv('./titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

In [13]:
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)
mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도 = {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도 = 0.7877


## 정리
> 정말 단순한 알고리즘으로 정확도가 무려 79% 가까지 나왔다. 정확도를 성능 평가 지표로 사용할 때는 매우 신중해야 한다. 특히, 정확도는 불균형한 레이블 값 분포에서 머신러닝 모델의 성능을 판단하기에 좋지 않다. 

# Mnist 데이터셋을 multi classification에서 binary classification으로 변경

In [19]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [15]:
# data loading
digits = load_digits()
# if digits number is 7, change to 1
y = (digits.target == 7).astype(int)
# split train and test dataset
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state = 11)

# check imbalanced label data
print('레이블 테스트 세트 크기 = ', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

레이블 테스트 세트 크기 =  (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64


In [18]:
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
predictions = fakeclf.predict(X_test)
accuracy = accuracy_score(y_test,predictions)
print('모든 예측을 0으로 하여도 정확도는 {0:.4f}이다.'.format(accuracy))

모든 예측을 0으로 하여도 정확도는 0.9000이다.


## 정리
> 불균형한 레이블 값을 가지는 데이터셋에서 정확도는 다른 성능 지표와 함께 쓰이는 것이 좋다. 

In [22]:
# 성능 평가 지표로 오차 행렬 활용하기
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, predictions)

array([[405,   0],
       [ 45,   0]])

In [26]:
'''
              prediction label 
               0     1     2
actual label 0 2

             1             1
             
             2 1           2
-> 나머지 모두 0
'''
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
confusion_matrix(y_true, y_pred)

array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]])